In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.constants import c, electron_mass as me, elementary_charge as qe, epsilon_0 as ep0
from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib.ticker as ticker

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Coupling Electron Tracing to Radiation Field Diagnostics

## QTNM Meeting 11/03/21

## Import Non-Relativistic Electron Tracking and Radiation modules

```python
# Add location of Single Electron Radiation
sys.path.insert(1, '/Users/tomgoffrey/dev/QTNM/SingleElectronRadiationQTNM/')
import SingleElectronRadiation as SER
from ford1991 import analytic_solution, rhs, solve
from utils import error_plot, calculate_omega
```

In [ ]:
# Add location of Single Electron Radiation
sys.path.insert(1, '/Users/tomgoffrey/dev/QTNM/SingleElectronRadiationQTNM/')
import SingleElectronRadiation as SER
from ford1991 import analytic_solution, rhs, solve
from utils import error_plot, calculate_omega

In [ ]:
%run config.py

In [ ]:
print(analytic_solution.__doc__)

In [ ]:
## Test values
tau = 0.015
B = 3.0
q = -1.0
m = 1.0

# Solve equation numerically
res = solve(5, b_field=B, charge=q, mass=m, tau=tau)

# Get analytic solution
x_soln, y_soln, vx_soln, vy_soln = analytic_solution(res.t, b_field=B, charge=q, mass=m, tau=tau)

# Plot result
error_plot(res.y[0], res.y[1], x_exact=x_soln, y_exact=y_soln,
          title='Electron Trajectory (Test values)', xlabel='x', ylabel='y')
plt.gcf().set_size_inches(3,3)
ax = plt.gca()
ax.set_yticks(plt.xticks()[0])
ax.set_ylim(ax.get_xlim())
ax.set_aspect(1)
plt.tight_layout()

In [ ]:
# Set-up detector, centred on (0, 0, 0.05)
x1 = 2.5e-2
x0 = -x1
y1 = 1e-2
y0 = -y1
z = 0.05
npixels = 20
# Detector Position Array
DPA = SER.CreateDetectorArray(npixels,x0,x1,y0,y1,z)

In [ ]:
# Set-up using ~QTNM values
B = 1
ke = 18.6
gamma_rel = 1 + 1e3 * ke * qe / me / c**2
beta_rel = np.sqrt(1 - 1 / gamma_rel**2)
vel = beta_rel * c # Initial velocity

# Calculate omega, with and without relativistic correction
omega = calculate_omega(B)
omega_rel = calculate_omega(B, energy=18.6)
print('omega = %.4E' % omega)
print('omega relativistic = %.4E' % omega_rel)

In [ ]:
# Get analytic solution at midpoint of rotation
time = np.pi / np.abs(omega)
x_soln, y_soln, vx_soln, vy_soln = analytic_solution(time, b_field=B, vel0=vel)

x = [x_soln, y_soln, vx_soln, vy_soln, 0.0]

# Also need acceleration
_, _, accx, accy, _ = rhs(time, x, omega, me, 0.0)

In [ ]:
# Daniel's code assumes negative x-direction B-field, so transform accordingly
EField = SER.CalcNonRelEFieldArray(DPA, time, [0, y_soln, x_soln], [0, accy, accx], npixels)
Emag = np.linalg.norm(EField,axis=2)   
p = plt.pcolormesh(Emag*1e6)
cbar = plt.colorbar(p)
cbar.set_label(r'$|\vec{E}|\;\mathrm{\mu V m^{-1}}$', rotation=90, fontsize=12)
plt.tight_layout()

In [ ]:
# Poynting flux magnitude - Does this need multiplying by pixel area?
Poynting = SER.CalcPoyntingVectorMagnitude(Emag)
p = plt.pcolormesh(Poynting*1e15)
plt.colorbar(p)
plt.tight_layout()

In [ ]:
## Actual values
## 6.26e-24
tau = qe * qe / (6.0 * np.pi * ep0 * c**3 * me)

# Solve equation numerically
res = solve(1, tau=tau, vel0=vel)

# Get analytic solution
x_soln, y_soln, vx_soln, vy_soln = analytic_solution(res.t, tau=tau, vel0=vel)

# Plot result
error_plot(res.y[0] * 1000, res.y[1] * 1000, x_exact=x_soln * 1000, y_exact=y_soln * 1000,
          title='Electron Trajectory', xlabel='x (mm)', ylabel='y (mm)')
plt.gcf().set_size_inches(3.25,3.25)
ax = plt.gca()
ax.set_yticks(plt.xticks()[0])
ax.set_ylim(ax.get_xlim())
ax.set_aspect(1)
plt.tight_layout()

In [ ]:
# Solve equation numerically for 100 rotations
n_rot = 100

res = solve(n_rot, tau=tau, vel0=vel)

# Get analytic solution
x_soln, y_soln, vx_soln, vy_soln = analytic_solution(res.t, tau=tau, vel0=vel)

In [ ]:
# Kinetic energy of electron as function of time
ke = 0.5 * me * (res.y[2]**2 + res.y[3]**2)
# Analytic solution
mu = omega**2 * tau
taue = - 2.0 * mu / (1.0 + tau**2)

error_plot(res.t * np.abs(omega), 1.0 - ke / ke[0], y_exact=1.0 - np.exp(taue * res.t),
           title='Fractional Electron Energy Loss', xlabel=r'$\omega_c t$', 
           ylabel=r'$\frac{\Delta T}{T_0}$')
plt.gcf().set_size_inches(5,3)
plt.title('Fractional Electron Energy Loss', y=1.08, fontsize=14)
plt.tight_layout()

In [ ]:
rad = np.sqrt(res.y[1]**2 + res.y[0]**2)
rad_exact = np.sqrt(y_soln**2 + x_soln**2)
error_plot(res.t * np.abs(omega), rad/rad[0] - 1.0, y_exact=rad_exact/rad_exact[0] - 1.0,
           xlabel='$\omega_c t$', ylabel=r'$\frac{\Delta r}{r(t=0)}$')
plt.title('Fractional Change in Radius', y=1.08, fontsize=14)
plt.tight_layout()

In [ ]:
frames_per_rot = 25
frames = n_rot * frames_per_rot
cadence = int(len(res.t) / frames)

# Only animate first 5 orbits
frames = int(frames / 20)
i = 0

time  = res.t[i]
Epos = [0.0, res.y[1,i], -res.y[0,i]]
_, _, accx, accy, _ = rhs(time, res.y[:,i], omega, me, tau)
Eacc = [0.0, accy, accx]

EField = SER.CalcNonRelEFieldArray(DPA, time, Epos, Eacc, npixels)
Emag = np.linalg.norm(EField,axis=2)   
Ex = EField[:,:,0]
Ey = EField[:,:,1]
Ez = EField[:,:,2]

fig, ax = plt.subplots(1,1)
Q = ax.quiver(DPA[:,:,0] * 100, DPA[:,:,1] * 100, Ex, Ey)
plt.xlabel('x(cm)')
plt.ylabel('y(cm)')
plt.title('Electric Field')
plt.tight_layout()

def update_plot(i, Q):
    j = i * cadence
    time  = res.t[j]
    Epos = [0.0, res.y[1,j], -res.y[0,j]]
    _, _, accx, accy, _ = rhs(time, res.y[:,j], omega, me, tau)
    Eacc = [0.0, accy, accx]

    EField = SER.CalcNonRelEFieldArray(DPA, time, Epos, Eacc, npixels)  
    Ex = EField[:,:,0]
    Ey = EField[:,:,1]
    Q.set_UVC(Ex, Ey)
    
    return Q,


anim = animation.FuncAnimation(fig, update_plot, fargs=(Q,), frames=frames, 
                               interval=200, blit=False)

fig.tight_layout()
plt.show()

In [ ]:
plt.rcParams['animation.embed_limit'] = 2**128
HTML(anim.to_jshtml())

In [ ]:
def calc_emag(j):
    time  = res.t[j]
    Epos = [0.0, res.y[1,j], -res.y[0,j]]
    _, _, accx, accy, _ = rhs(time, res.y[:,j], omega, me, tau)
    Eacc = [0.0, accy, accx]
    EField = SER.CalcNonRelEFieldArray(DPA, time, Epos, Eacc, npixels)
    Emag = np.linalg.norm(EField,axis=2)   

    return Emag


Emin = 1e100
Emax = 0.0

for i in range(frames):
    Emag = calc_emag(i*cadence)
    Emin = min(np.min(Emag), Emin)
    Emax = max(np.max(Emag), Emax)

Emag = calc_emag(0)
fig, ax = plt.subplots(1,1)
data = plt.pcolormesh(DPA[:,:,0]*100, DPA[:,:,1]*100, Emag,
                      vmin=Emin, vmax=Emax, shading='gouraud')

def fmt(x, pos):
    a, b = '{:1.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

cb = plt.colorbar(data, format=ticker.FuncFormatter(fmt))
cb.ax.tick_params(labelsize=10)

plt.xlabel('x(cm)')
plt.ylabel('y(cm)')
plt.title('Electric Field Magnitude', y=1.08)

plt.tight_layout()

def update_plot(i, data):
    j = i * cadence
    Emag = calc_emag(j)
    data.set_array(Emag)
    
    return data,


anim = animation.FuncAnimation(fig, update_plot, fargs=(data,), frames=frames, 
                               interval=200, blit=False)

fig.tight_layout()
plt.show()

In [ ]:
HTML(anim.to_jshtml())